In [3]:
import sys

datasetName = sys.argv[1] #'airports' # hospital, ncvoter, inspection
import pandas as pd
import argparse
import logging 
import sys
sys.path.append('../../REEs_model')
sys.path.append('../../')
sys.path.append('../REEs_model')
from interestingnessFixedEmbeds import *
from predicateAgentInterestingness import Predicate
from REEs_model.utils import *

data_prefix = '../../REEs_model_data/revision/labeled_data_400/'
num_of_instances = 400

rules_set_path = os.path.join(data_prefix, datasetName, datasetName + '_rules.txt')
#predicates_file = os.path.join(data_prefix, datasetName, datasetName + '_all_predicates.txt')
predicates_file = os.path.join(data_prefix, datasetName, 'train', 'all_predicates.txt')
tokens_file = os.path.join(data_prefix, datasetName, 'train', 'tokenVobs.txt')
token_embeddings_file = os.path.join(data_prefix, datasetName, 'train', 'tokenEmbedds.pkl')

rules_pair_ids_path = os.path.join(data_prefix, datasetName, 'labeled_data.txt')
train_pair_ids_path = os.path.join(data_prefix, datasetName, 'train', 'train.csv')
valid_pair_ids_path = os.path.join(data_prefix, datasetName, 'train', 'valid.csv')
test_pair_ids_path = os.path.join(data_prefix, datasetName, 'train', 'test.csv')

import shutil
# copy rule file
shutil.copy(rules_set_path, os.path.join(data_prefix, datasetName, 'train', 'rules.txt'))

'../../REEs_model_data/revision/labeled_data_400/airports/train/rules.txt'

In [4]:
# split training, validation and testing data
#data_file = sys.argv[1]
training_ratio = 0.8 #float(sys.argv[2])

#rule_pair_set_path = os.path.join(data_prefix, datasetName, 'labeled_data.txt')

rules_pair_ids_set = pd.read_csv(rules_pair_ids_path, delimiter=" ")

schemas = ['left_id', 'right_id', 'label'] #list(rules_pair_ids_set.columns)
#rules_pair_ids_set
rules_pair_ids_set = rules_pair_ids_set.values

pairs_num = len(rules_pair_ids_set)

scripts = np.arange(pairs_num) 
np.random.seed(20)
np.random.shuffle(scripts)
scripts = scripts[:num_of_instances]
pairs_num = num_of_instances
validation_ratio = (1 - training_ratio) / 2
training_num, validation_num = int(training_ratio * pairs_num), int(validation_ratio * pairs_num)
scripts_train = scripts[:training_num]
scripts_valid = scripts[training_num: training_num + validation_num]
scripts_test = scripts[training_num + validation_num:]

train, valid, test = rules_pair_ids_set[scripts_train], rules_pair_ids_set[scripts_valid], rules_pair_ids_set[scripts_test]
train_df = pd.DataFrame(train, columns=schemas)
valid_df = pd.DataFrame(valid, columns=schemas)
test_df = pd.DataFrame(test, columns=schemas)

train_df.to_csv(train_pair_ids_path, index=False)
valid_df.to_csv(valid_pair_ids_path, index=False)
test_df.to_csv(test_pair_ids_path, index=False)



In [5]:
os.environ["WANDB_DISABLED"] = "true"

In [6]:


rules_set = pd.read_csv(rules_set_path)
rules_set = rules_set.values

# # train
# option = 'train'
# rule_pair_set_path = os.path.join(data_prefix, datasetName, 'labeled_data.txt_' + option + '.csv')
# train_data = pd.read_csv(rule_pair_set_path, delimiter=",")

# # valid
# option = 'valid'
# rule_pair_set_path = os.path.join(data_prefix, datasetName, 'labeled_data.txt_' + option + '.csv')
# valid_data = pd.read_csv(rule_pair_set_path, delimiter=",")

# # test
# option = 'test'
# rule_pair_set_path = os.path.join(data_prefix, datasetName, 'labeled_data.txt_' + option + '.csv')
# test_data = pd.read_csv(rule_pair_set_path, delimiter=",")







In [7]:
rules_set[:, 0]

array(['airports(t0) ^ airports(t1) ^ t0.iso_region == t1.iso_region ^ t0.latitude_deg == t1.latitude_deg ^ t0.local_code == t1.local_code  ->  t0.longitude_deg == t1.longitude_deg',
       'airports(t0) ^ airports(t1) ^ t0.longitude_deg == t1.longitude_deg ^ t0.municipality == t1.municipality  ->  t0.iso_country == t1.iso_country',
       'airports(t0) ^ airports(t1) ^ t0.latitude_deg == t1.latitude_deg  ->  t0.iata_code == t1.iata_code',
       'airports(t0) ^ airports(t1) ^ t0.latitude_deg == t1.latitude_deg ^ t0.municipality == t1.municipality  ->  t0.type == t1.type',
       'airports(t0) ^ airports(t1) ^ t0.iso_country == BR ^ t0.municipality == t1.municipality  ->  t0.iata_code == t1.iata_code',
       'airports(t0) ^ airports(t1) ^ t0.gps_code == t1.gps_code ^ t0.longitude_deg == t1.longitude_deg  ->  t0.iso_country == t1.iso_country',
       'airports(t0) ^ airports(t1) ^ t0.elevation_ft == t1.elevation_ft ^ t0.longitude_deg == t1.longitude_deg  ->  t0.type == t1.type',
      

In [8]:
# remove Relation(t0/1) and add Relation into Relation.t0/1
def revisePredicate(predicate, datasetName):
    if predicate[:len(datasetName)] == datasetName:
        return None
    p = predicate.replace('t0', datasetName + '.t0')
    p = p.replace('t1', datasetName + '.t1')
    return p
    

# extract all predicates
def extractPredicates(rules):
    predicates = defaultdict(int)
    for rule in rules:
        lhs, rhs = rule.split('->')[0], rule.split('->')[1]
        #print(rhs)
        lhs = lhs.split('^')
        predicates[rhs.strip()] += 1
        for p in lhs:
            predicates[p.strip()] += 1
    return predicates

predicates = extractPredicates(rules_set[:, 0])
# revise predicates
predicates_ = []
for p in predicates:
    p_ = revisePredicate(p, datasetName)
    if p_ != None:
        predicates_.append(p_)

predicates = predicates_

# save
f = open(predicates_file, 'w')
for p in predicates:
    f.write(p)
    f.write('\n')
f.close()

In [9]:
s = 't0.dflkdlfk = t1.dgfklfkgl'
s = s.replace('t0', datasetName + '.t0')
s = s.replace('t1', datasetName + '.t1')
s

'airports.t0.dflkdlfk = airports.t1.dgfklfkgl'

In [10]:
predicates

['airports.t0.longitude_deg == airports.t1.longitude_deg',
 'airports.t0.iso_region == airports.t1.iso_region',
 'airports.t0.latitude_deg == airports.t1.latitude_deg',
 'airports.t0.local_code == airports.t1.local_code',
 'airports.t0.iso_country == airports.t1.iso_country',
 'airports.t0.municipality == airports.t1.municipality',
 'airports.t0.iata_code == airports.t1.iata_code',
 'airports.t0.type == airports.t1.type',
 'airports.t0.iso_country == BR',
 'airports.t0.gps_code == airports.t1.gps_code',
 'airports.t0.elevation_ft == airports.t1.elevation_ft',
 'airports.t0.name == airports.t1.name',
 'airports.t0.wikipedia_link == airports.t1.wikipedia_link',
 'airports.t0.scheduled_service == airports.t1.scheduled_service',
 'airports.t0.iso_country == US',
 'airports.t0.keywords == airports.t1.keywords',
 'airports.t0.home_link == airports.t1.home_link',
 'airports.t0.continent == airports.t1.continent']

In [11]:
# generate tokenVobs
sys.path.append('../')
from utils import generateVobs
tokenVobs = generateVobs(predicates_file)

vobs_file = os.path.join(data_prefix, datasetName, 'train', 'tokenVobs.txt')
f = open(vobs_file, 'w')
for k, v in tokenVobs.items():
    f.write(str(k) + " " + str(v))
    f.write("\n")
f.close()


Parse predicate : airports.t0.longitude_deg == airports.t1.longitude_deg, ['t0', 'airports___longitude_deg', '==', 't1', 'airports___longitude_deg']
Parse predicate : airports.t0.iso_region == airports.t1.iso_region, ['t0', 'airports___iso_region', '==', 't1', 'airports___iso_region']
Parse predicate : airports.t0.latitude_deg == airports.t1.latitude_deg, ['t0', 'airports___latitude_deg', '==', 't1', 'airports___latitude_deg']
Parse predicate : airports.t0.local_code == airports.t1.local_code, ['t0', 'airports___local_code', '==', 't1', 'airports___local_code']
Parse predicate : airports.t0.iso_country == airports.t1.iso_country, ['t0', 'airports___iso_country', '==', 't1', 'airports___iso_country']
Parse predicate : airports.t0.municipality == airports.t1.municipality, ['t0', 'airports___municipality', '==', 't1', 'airports___municipality']
Parse predicate : airports.t0.iata_code == airports.t1.iata_code, ['t0', 'airports___iata_code', '==', 't1', 'airports___iata_code']
Parse predica

In [12]:
# embeddings
from transformers import BertTokenizer, BertModel
import torch



tokenizer = BertTokenizer.from_pretrained('distilbert-base-uncased')
model = BertModel.from_pretrained('distilbert-base-uncased')

tokenVobs = []
for line in open(tokens_file):
    tokenVobs.append(line.split()[0].strip())

print(tokenVobs[:5])

#inputs = tokenizer('Airport t(1) ^ name', return_tensors='pt')
inputs = tokenizer(tokenVobs[0], return_tensors='pt')
outputs = model(**inputs)

pooler_output = outputs.pooler_output




The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing BertModel: ['distilbert.transformer.layer.1.ffn.lin2.bias', 'distilbert.transformer.layer.0.ffn.lin1.weight', 'vocab_layer_norm.bias', 'distilbert.transformer.layer.1.attention.v_lin.weight', 'distilbert.transformer.layer.4.attention.q_lin.weight', 'distilbert.transformer.layer.5.ffn.lin2.weight', 'distilbert.transformer.layer.0.attention.k_lin.weight', 'distilbert.transformer.layer.1.attention.k_lin.bias', 'distilbert.transformer.layer.2.attention.v_l

['PAD', 't0', 't1', '==', 'airports']


In [13]:
print(pooler_output.shape)



torch.Size([1, 768])


In [14]:
tokenEmbedds = dict()
for token in tokenVobs:
    inputs = tokenizer(token, return_tensors='pt')
    outputs = model(**inputs)
    pooler_output = outputs.pooler_output
    print(pooler_output.shape)
    tokenEmbedds[token] = np.array(pooler_output.detach().numpy())


torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])


In [15]:
import pickle as pkl

with open(token_embeddings_file, 'wb') as f:
    pkl.dump(tokenEmbedds, f)
